# Randomized Movement of Enemies

Our feature we wanted to import from the platformer game is the randomized movement of characters. When we go to the platformer game, we are introduced to Goombas, these tiny little mushroom creatures you have to stomp on or they kill you on collision. If you pay close attention to the movement of the Goombas, there is a set space where the Goombas are allowed to move, but instead of them changing direction when they reach the boundary of that set walking space, they change direction in random intervals *inside* the walking space. 

This allows for a more natural and less rigid/robotic movement for our enemies. Instead of being able to predict when the enemy changes direction (helpful for more complicated levels with more strategizing needed; we don't want to overwhelm our players), we can use this randomized movement to add another layer of suspense into our game (helpful when our level is fairly simple at a base level and we need a little something to spice it up).


Here are a few combined snippets of the code for the goombas seen in Mr. Mortensen's original platformer game. Here we see the code that describes the goombas' movement throughout the game. 

```js
        //[...]
        

        //Initial Position of Goomba
        this.x = xPercentage * GameEnv.innerWidth;

        //Access in which a Goomba can travel    
        this.minPosition = minPosition * GameEnv.innerWidth;
        this.maxPosition = this.x + xPercentage * GameEnv.innerWidth;


        //Define Speed of Enemy
        if (["easy", "normal"].includes(GameEnv.difficulty)) {
            this.speed = this.speed * Math.floor(Math.random() * 1.5 + 2);
        } else if (GameEnv.difficulty === "hard") {
            this.speed = this.speed * Math.floor(Math.random() * 3 + 3);
        } else {
            this.speed = this.speed * 5
        }


        //[...]


        // Every so often change direction
        switch(GameEnv.difficulty) {
            case "normal":
                if (Math.random() < 0.005) this.speed = -this.speed;
                break;
            case "hard":
                if (Math.random() < 0.01) this.speed = -this.speed;
                break;
            case "impossible":
                if (Math.random() < 0.02) this.speed = -this.speed;
                break;
        }


        //[...]


        // Move the enemy
        this.x -= this.speed;
        // Randomly trigger a jump (increased probability)
        if (Math.random() < 0.1) { // Adjust the probability as needed
            this.jump();
        }
```
First, we see the goomba's initial position, described as `this.x = xPercentage * GameEnv.innerWidth;`. Next, we define the minimum and maximum positions that the goomba can travel in. Then, we define the speeds we want for the difficulty of the game level. 

Now we get to the part we're looking for. The method `switch(GameEnv.difficulty)` provides us with the consequences for different cases. Here, we can see if the level difficulty of the game is "normal", there is a less than 0.5% chance that the Goomba switches directions (direction change being `this.speed = -this.speed`). This percentage keeps increasing based on the case. 

The last bit of the code simply gives us the actual movement of the Goomba and a random jump trigger. 

However, this is not the only way to accomplish what we're looking for. Instead of making a separate file for the Goombas, we created a function for the movement inside our level. Here is a snippet of our code, with commentary:

```js
function setupRandomMotion(spriteData, yOffset = 0) { // initialises a simple left‑right “wandering” behaviour for one sprite (an element on the page).
      spriteData.direction = { x: Math.random() < 0.5 ? -1 : 1 };
      // direction.x is either –1 (moving left) or +1 (moving right), chosen randomly. the "Math.random() < 0.5" makes it so that if the value of Math.random is < 0.5, the enemy continues left. If not, it continues right. 
      spriteData.speed = 5;
      // speed is hard‑coded to 5 px per tick .
      spriteData.INIT_POSITION = {
        x: Math.random() * (width - 200) + 100,
      // The x‑value is random but clamped so the sprite starts 100 px in from each edge .
        y: height / 2 + yOffset
      // The y‑value is halfway down the canvas (height / 2) plus any extra offset passed in.
      };


      setInterval(() => { // This function is called every 100 ms to update the sprite’s position.
        spriteData.INIT_POSITION.x += spriteData.direction.x * spriteData.speed;

        if (spriteData.INIT_POSITION.x < 0) {
          spriteData.INIT_POSITION.x = 0;
          spriteData.direction.x = 1;
        }
        // If the sprite hits the left edge, lock it to x = 0 and force direction → right. 
        if (spriteData.INIT_POSITION.x > width - 100) {
          spriteData.INIT_POSITION.x = width - 100;
          spriteData.direction.x = -1;
        }
        // If it reaches width – 100, lock it there and force direction → left. 
        // These two if statements ensures that the sprite is always fully on‑screen.


        if (Math.random() < 0.03) {
          spriteData.direction.x *= -1;
        }

        // On about 3 % of ticks, the sprite turns around randomly, so the player can't predict its movement.

        const spriteElement = document.getElementById(spriteData.id);
        if (spriteElement) {
          spriteElement.style.transform = spriteData.direction.x === -1 ? "scaleX(-1)" : "scaleX(1)";
          // scaleX(−1) flips the image so it faces the way it’s moving.
          spriteElement.style.left = spriteData.INIT_POSITION.x + 'px';
          // left sets the sprite’s pixel position.
          spriteElement.style.top = spriteData.INIT_POSITION.y + 'px';
          // top sets the sprite’s pixel position.
        }
      }, 100);
    }
```